# Evaluate vector databases

In [1]:
from typing import Dict, List
from langchain_core.documents.base import Document

## Load evaluation dataset and docs

In [2]:
from langchain_community.document_loaders import NotionDBLoader
import tomllib

In [3]:
with open('../.tokens.toml', 'rb') as f:
    _TOKENS = tomllib.load(f)

with open('../.notion_databases.toml', 'rb') as f:
    _DATABASES_NOTION = tomllib.load(f)

with open('../.config.toml', 'rb') as f:
    _CONFIGS = tomllib.load(f)


In [4]:
# def load_notion_dbs(dbs, id):
#     loader = NotionDBLoader(
#         integration_token=_TOKENS['notion'],
#         database_id=dbs[id],
#         request_timeout_sec=300,  # optional, defaults to 10
#     )
#     data = loader.load()
#     return data

In [5]:
# %%time
# from concurrent.futures import ThreadPoolExecutor

# te = ThreadPoolExecutor()
# results = list(te.map(lambda x: load_notion_dbs(_DATABASES_NOTION, x), _DATABASES_NOTION.keys()))

# docs_from_notion: Dict[str, List[Document]] = dict(zip(_DATABASES_NOTION.keys(), results))

# # optional pickle step so we don't need to query notionDB again
# import pickle

# with open('../data/notion_offline.pkl', 'wb') as f:
#     pickle.dump(docs_from_notion, f)

In [6]:
import pickle
with open('../data/notion_offline.pkl', 'rb') as f:
    docs_from_notion = pickle.load(f)

In [7]:
docs_from_notion['写作'][0].metadata

{'date': {'start': '2013-10-26', 'end': '2013-10-26', 'time_zone': None},
 'name': '2013-OCT-26 师说',
 'tags': ['日常记趣'],
 'id': '273ea76f-a35c-474e-bfe0-41a3daae5c96'}

In [8]:
def process_property(docs_from_notion: Dict[str, List[Document]]) -> List[Document]:
    docs_list = list()
    
    for db_name, docs in docs_from_notion.items():
        for doc in docs:
            # because our data are gathered from multiple databases
            # we are going to throw the database names as one property
            # into the docs' metadata field
            # and return as a list
            doc.metadata['source'] = db_name

            # change dates into YYYYMMDD int format to allow GT/LT/EQ comparison
            if 'date' in doc.metadata:
                if 'start' in doc.metadata['date']:
                    doc.metadata['date_start'] = int(doc.metadata['date']['start'].replace("-", ""))
                if 'end' in doc.metadata['date'] and doc.metadata['date']['end']:
                    doc.metadata['date_end'] = int(doc.metadata['date']['end'].replace("-", ""))
                    
                del doc.metadata['date']

            if 'tags' in doc.metadata:
                doc.metadata['tags'] = ", ".join(doc.metadata['tags'])
                

        docs_list.extend(docs)
        
    return docs_list

In [9]:
docs_from_notion = process_property(docs_from_notion)
docs_from_notion[0].metadata

{'name': '2013-OCT-26 师说',
 'tags': '日常记趣',
 'id': '273ea76f-a35c-474e-bfe0-41a3daae5c96',
 'source': '写作',
 'date_start': 20131026,
 'date_end': 20131026}

In [10]:
# Let an automated process takes care the rest
from langchain_community.vectorstores.utils import filter_complex_metadata
docs_from_notion = filter_complex_metadata(docs_from_notion)

In [11]:
from langchain_community.vectorstores import Redis

## Testing ideas

* use langchain.text_splitterRecursiveCharacterTextSplitter
* storage: test 2 vector databses: Redis, Superbase

In [12]:
# Presumably docs in NotionDB fits more with MarkdownHeaderTextSplitter
# however, most of the documents in my personal databases don't have such header-text structure
# and they are not important for my use cases (I won't ask it to reason on a specific section 
# in a doc). Thus I'll use the regular RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
chunk_params = {
    # 1 Chinese characters = 2 english character
    # most paragraphs/sections are within 500 Chinese words/chars
    'chunk_size': 1000, 
    'chunk_overlap': 250,
}

rc_splitter = RecursiveCharacterTextSplitter(**chunk_params)

splits = rc_splitter.split_documents(docs_from_notion)
len(docs_from_notion), len(splits)

(225, 1136)

In [14]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=_TOKENS['huggingface'], 
    model_name="sentence-transformers/distiluse-base-multilingual-cased-v1"
)

In [15]:
metadata_set = set()

for x in [x.metadata for x in docs_from_notion]:
    metadata_set = metadata_set.union(list(x.keys()))

metadata_set

{'author', 'date_end', 'date_start', 'id', 'name', 'source', 'tags'}

In [29]:
# make sure we defined the schema for all metadata in _CONFIG file
assert metadata_set == \
    set([x['name'] for x in _CONFIGS['redis_schema']['text']] + [x['name'] for x in _CONFIGS['redis_schema']['numeric']])

In [16]:
docs_from_notion[0].metadata

{'name': '2013-OCT-26 师说',
 'tags': '日常记趣',
 'id': '273ea76f-a35c-474e-bfe0-41a3daae5c96',
 'source': '写作',
 'date_start': 20131026,
 'date_end': 20131026}

In [17]:
!redis-stack-server # init redis-stack server

Starting redis-stack-server, database path /opt/homebrew/var/db/redis-stack
2581:C 18 Feb 2024 16:57:47.406 * oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
2581:C 18 Feb 2024 16:57:47.406 * Redis version=7.2.4, bits=64, commit=d2c8a4b9, modified=0, pid=2581, just started
2581:C 18 Feb 2024 16:57:47.406 * Configuration loaded
2581:M 18 Feb 2024 16:57:47.406 * Increased maximum number of open files to 10032 (it was originally set to 4864).
2581:M 18 Feb 2024 16:57:47.406 * monotonic clock: POSIX clock_gettime
                _._                                                  
           _.-``__ ''-._                                             
      _.-``    `.  `_.  ''-._           Redis 7.2.4 (d2c8a4b9/0) 64 bit
  .-`` .-```.  ```\/    _.,_ ''-._                                  
 (    '      ,       .-`  | `,    )     Running in standalone mode
 |`-._`-...-` __...-.``-._|'` _.-'|     Port: 6379
 |    `-._   `._    /     _.-'    |     PID: 2581
  `-._    `-._  `-./  _.-'    _.-'    

In [18]:
Redis.drop_index(
    redis_url=_CONFIGS['redis_url'],
    index_name=_CONFIGS['index_name'], 
    delete_documents=True
)

True

In [19]:
%%time

# Redis supports default "tag" fields alongside with "text" and "numeric"
# looks like a better match for "tags" property at the first glance
# but we'll classify it as "text" anyway because the to give consistency of 
# how downstream self-query writes filter queries.
# ref: https://redis.io/docs/interact/search-and-query/advanced-concepts/tags

vectorstore = Redis.from_documents(
    documents=docs_from_notion,
    embedding=embeddings,
    redis_url=_CONFIGS['redis_url'],
    index_name=_CONFIGS['index_name'],
    index_schema=_CONFIGS['redis_schema'],
)

`index_schema` does not match generated metadata schema.
If you meant to manually override the schema, please ignore this message.
index_schema: {'text': [{'name': 'author'}, {'name': 'id'}, {'name': 'name'}, {'name': 'source'}, {'name': 'tags'}], 'numeric': [{'name': 'date_start'}, {'name': 'date_end'}]}
generated_schema: {'text': [{'name': 'name'}, {'name': 'tags'}, {'name': 'id'}, {'name': 'source'}], 'numeric': [{'name': 'date_start'}, {'name': 'date_end'}], 'tag': []}



CPU times: user 132 ms, sys: 27.9 ms, total: 159 ms
Wall time: 1.14 s


In [20]:
!rvl index info -i notiondb



Index Information:
╭──────────────┬────────────────┬──────────────────┬─────────────────┬────────────╮
│ Index Name   │ Storage Type   │ Prefixes         │ Index Options   │   Indexing │
├──────────────┼────────────────┼──────────────────┼─────────────────┼────────────┤
│ notiondb     │ HASH           │ ['doc:notiondb'] │ []              │          0 │
╰──────────────┴────────────────┴──────────────────┴─────────────────┴────────────╯
Index Fields:
╭────────────────┬────────────────┬─────────┬────────────────┬────────────────┬────────────────┬────────────────┬────────────────┬────────────────┬─────────────────┬────────────────╮
│ Name           │ Attribute      │ Type    │ Field Option   │ Option Value   │ Field Option   │ Option Value   │ Field Option   │   Option Value │ Field Option    │ Option Value   │
├────────────────┼────────────────┼─────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────┼─────────────────┼──────────────

In [21]:
from langchain.vectorstores.redis import RedisFilter, RedisNum, RedisText

# form comparator logic
# ref: https://github.com/langchain-ai/langchain/blob/d7c26c89b2d4f5ff676ba7c3ad4f9075d50a8ab7/libs/community/langchain_community/vectorstores/redis/filters.py#L313

f = RedisText("author") % "*冯友兰*"
vectorstore.similarity_search_with_score('谁说过陌生贵己？', filter=f, k=3)

Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']


[(Document(page_content='中国哲学的精神\n中国文化的精神基础是\n伦理\n（特别是儒家伦理）而不是宗教\n中国人不以宗教观念和宗教活动为生活中最重要、最迷人的部分。这和其他主要文化（以寺院、僧侣为主导）有根本的重要的不同\n中国人在哲学里满足了他们对超乎现世的追求，也在哲学里表达、欣赏和体验超道德价值\n人们习惯于说中国有三教：儒教、道教、佛教。但其实儒教比起宗教更是一种伦理、哲学或者文化；作为一个哲学学派的道家（顺其自然）和以研究方术避免死亡的道教（反乎自然）也有区别；同理，作为哲学的佛学比宗教的佛教在社会上的影响大得多\n为道 ≠ 为学。在中国哲学的传统中，哲学并不在于增加积极的、实用的知识，而在于提高心灵的境界，获得高于道德价值的价值\n中国的哲学既是出世的也是入世的\n“不离日用常行内，直到先天未画前”：中国哲学寻求解决的问题，是把入世和出世这两个对立的命题统一成一个合命题。它是最理想主义的，也是最现实主义的\n“内圣外王”：中国哲学中，如果一个人不仅在理论而且在行动上能够完成这种统一，他就是圣人，也是最适宜为王之人。这和柏拉图“哲学家-王”的理论很有相似之处\n中国的哲学和政治思想无法分开：儒家认为，处理日常的人伦世务，不是圣人的分外之事。处理世务，正是圣人人格完全发展的实质所在。他不仅作为社会的公民，而且也作为孟子所谓“天民”，来执行这个任务。正如柏拉图的《理想国》既代表了他的整个哲学，同时也是他的政治思想\n中国的哲学家以身载道：这个哲学家遵守他的哲学信念而生活，他的生活是他的哲学的组成部分。中国哲学家都是不同程度的苏格拉底，对于他们，哲学从来就不是为人类认识摆设的观念模式，而是内在与他的行动的箴言体系；在极端情况下，他的哲学简直可以说是他的传记\n中国哲学的阐述多依靠不明晰的比喻论证，而没有正式哲学著作\n中国哲学家看来，研究哲学不是一种职业，因此中国没有职业哲学家\n研究中国的哲学，只有看这些人的语录或者写给学生、朋友的信。这些片段来自于他一生的各个时期，语录也不只是一人所记。所以它们不相联系，有时甚至互相矛盾\n中国哲学家惯用名言隽语、比喻例证来表达自己的思想。这种表达明晰不足而暗示有余。明晰和暗示是不可兼得的，正如一种表达越是散文化就越缺乏诗意；而富于暗示的倾向性是中国一切艺术的理想（“言有尽而意无穷”）\n郭象是《庄

In [22]:
f = RedisText("name") % "*三岛由纪夫*"
vectorstore.similarity_search_with_score('清显与本多', filter=f, k=3)

Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']


[(Document(page_content='P7\n从坡道走下来的，原来是一个年轻人。他前后挑着一担粪桶，头缠一条肮脏的手巾，有一张漂亮的红润的脸，和一双炯炯有神的眼，他迈着稳重的脚步从坡道上走了下来。他是个清厕夫——掏大粪的人。年轻人脚蹬胶底布袜子，身穿\n藏青色紧腿裤\n。五岁的我，以异常的目光看了看这个姿影。它的意义还不明确，然而这是一种力量的最先的启示、是一种阴暗的不可思议的呼声在召唤着我。第一次显现在清厕夫身影上，是具有寓意的。因为粪尿是大地的象征。召唤我的东西，无疑是伊耶那美命神的带有恶意的爱。\n我预感到这世上存在一种火辣辣的刺痛似的欲望。我一边抬头仰望着肮脏的小伙子的身影，一边被一种“\n我想成为他\n”的欲望、“\n但愿我就是他\n”的欲望，紧紧地纠缠着。让人很明显地想到这种欲望有两个重点：一个重点是他的\n藏青色紧腿裤\n，另一个重点是他的职业。藏青色紧腿裤把他的下半身的轮廓清楚地勾勒了出来。它使我联想起仿佛有一种东西在优美地活动着，正在向我走近过来。我对这条紧腿裤竟产生一股无可名状的倾倒。究竟为什么，我也不明白。\n他的职业——这时候，我刚开始懂事，就像其他孩子向往长大当陆军大将的心态一样，我的脑海里就浮现出“想当清厕夫”的憧憬。憧憬的原因可以归咎那条藏青色的紧腿裤，但绝不仅仅在此。这主题本身在我的内心里不断强化，发展，让人看到了一种特异的展现。\n之所以如此，是因为我对他的职业感受到一种对于尖锐的悲哀、彻身透骨的悲哀的憧憬似的东西。一种极其感觉意义上的“悲剧式的东西”。从他的职业产生一种“挺身”的感觉、一种草率的感觉、一种对危险亲近的感觉，以及虚无和活力惊人混合的感觉。\n这些感觉洋溢出来，向五岁的我逼将过来，把我俘虏了。也许我误解了清厕夫这种职业。也许我把从别人那里听来的别的什么职业，以他的服装误认为他的职业，硬把它纳入他的职业里，否则就无法加以解释。\n为什么呢？因为同这种情绪一样的主题，不久就转移到花车的司机和地铁检票员的身上，他们强烈地使我感受到一种我所不了解的、可以说是我永远从那里被排除在外的“悲剧性的生活”。特别是地铁剪票员的情况，当时地铁内弥漫着橡胶般的、薄荷般的气味，与他的绿色制服胸前的成排金扣互相结合，很容易促使我联想起“悲剧性的东西”来。\n\n\nP21\n渐渐隐约地传来的打夯歌的悲调，贯穿这无序的节日的嘈杂，告知

In [23]:
f = (RedisNum("date_start") > 20201231) & (RedisNum("date_start") < 20211231)
vectorstore.similarity_search_with_score("", filter=f, k=2)

Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']


[(Document(page_content='一个人可以想象的最崇高的生活方式，就是将他自己的身体挡在荒芜的战场和可爱的家园之间。当然，你也知道，这些话不是我说的。基本的真理不会改变，一旦一个有洞察力的人表达了它们，那么无论这个世界怎么改变，都没有必要再对它们做出更改。它们是不变的，无论何时何地，无论对于谁，对于哪个国家来说都是真的。\n\n横行在大街上的小流氓是一个外部症状，表明那个文明已经身患重病：其公民只知道称颂“权利”的神话，却忘记了他们的责任。由这样的公民组成的国家是不可能国运长久的。', metadata={'id': 'doc:notiondb:b37803b5579a405abe698a39ee98cb72', 'author': '【美】罗伯特·海因莱因', 'name': '星船伞兵 【美】罗伯特·海因莱因', 'source': '读书笔记（文学）', 'tags': '小说, 科幻', 'date_start': '20210911', 'date_end': None}),
  0.9743),
 (Document(page_content='Rites of Passage at $100,000+\nThe insider’s lifetime guide to executive job-changing and faster career progress | John Lucht\n\nSqueaky wheel gets the grease\nRealizing that you can move effectively if and when you decide to do so, you’ll feel a lot freer to have period frank discussion with your recent superiors, letting them know that you feel ready to step ahead, and drawing out their unvarnished reactions to your self-appraisal.\nThe self-confident person asking company’s timetable for his own p

In [24]:
f = RedisText("name") % "*三岛由纪夫*"
vectorstore.max_marginal_relevance_search('清显与本多', filter=f, k=3)

Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'n

[Document(page_content='P7\n从坡道走下来的，原来是一个年轻人。他前后挑着一担粪桶，头缠一条肮脏的手巾，有一张漂亮的红润的脸，和一双炯炯有神的眼，他迈着稳重的脚步从坡道上走了下来。他是个清厕夫——掏大粪的人。年轻人脚蹬胶底布袜子，身穿\n藏青色紧腿裤\n。五岁的我，以异常的目光看了看这个姿影。它的意义还不明确，然而这是一种力量的最先的启示、是一种阴暗的不可思议的呼声在召唤着我。第一次显现在清厕夫身影上，是具有寓意的。因为粪尿是大地的象征。召唤我的东西，无疑是伊耶那美命神的带有恶意的爱。\n我预感到这世上存在一种火辣辣的刺痛似的欲望。我一边抬头仰望着肮脏的小伙子的身影，一边被一种“\n我想成为他\n”的欲望、“\n但愿我就是他\n”的欲望，紧紧地纠缠着。让人很明显地想到这种欲望有两个重点：一个重点是他的\n藏青色紧腿裤\n，另一个重点是他的职业。藏青色紧腿裤把他的下半身的轮廓清楚地勾勒了出来。它使我联想起仿佛有一种东西在优美地活动着，正在向我走近过来。我对这条紧腿裤竟产生一股无可名状的倾倒。究竟为什么，我也不明白。\n他的职业——这时候，我刚开始懂事，就像其他孩子向往长大当陆军大将的心态一样，我的脑海里就浮现出“想当清厕夫”的憧憬。憧憬的原因可以归咎那条藏青色的紧腿裤，但绝不仅仅在此。这主题本身在我的内心里不断强化，发展，让人看到了一种特异的展现。\n之所以如此，是因为我对他的职业感受到一种对于尖锐的悲哀、彻身透骨的悲哀的憧憬似的东西。一种极其感觉意义上的“悲剧式的东西”。从他的职业产生一种“挺身”的感觉、一种草率的感觉、一种对危险亲近的感觉，以及虚无和活力惊人混合的感觉。\n这些感觉洋溢出来，向五岁的我逼将过来，把我俘虏了。也许我误解了清厕夫这种职业。也许我把从别人那里听来的别的什么职业，以他的服装误认为他的职业，硬把它纳入他的职业里，否则就无法加以解释。\n为什么呢？因为同这种情绪一样的主题，不久就转移到花车的司机和地铁检票员的身上，他们强烈地使我感受到一种我所不了解的、可以说是我永远从那里被排除在外的“悲剧性的生活”。特别是地铁剪票员的情况，当时地铁内弥漫着橡胶般的、薄荷般的气味，与他的绿色制服胸前的成排金扣互相结合，很容易促使我联想起“悲剧性的东西”来。\n\n\nP21\n渐渐隐约地传来的打夯歌的悲调，贯穿这无序的节日的嘈杂，告知这